In [8]:
# ---------------import cell------------------
import numpy as np
import random
import math
import binascii
import copy

In [9]:
RC = ['00000001', '00000010', '00000100', '00001000', '00010000', '00100000', '01000000', '10000000', '00011011',
      '00110110']

mixColumn = [['02', '03', '01', '01'],
             ['01', '02', '03', '01'],
             ['01', '01', '02', '03'],
             ['03', '01', '01', '02']]

inv_mixColumn = [['0e', '0b', '0d', '09'],
                 ['09', '0e', '0b', '0d'],
                 ['0d', '09', '0e', '0b'],
                 ['0b', '0d', '09', '0e']]

def xor(a, b):
    return (a and not b) or (not a and b)


def binary2string(decryptedData):
    strDecryptedData = ''
    for i in decryptedData:
        if i == 1:
            strDecryptedData += '1'
        elif i == 0:
            strDecryptedData += '0'


def hexadecimal_to_8bits_binary(hex_string):
    hex_number = int(hex_string, 16)
    binary_representation = bin(hex_number)[2:]  # Convert to binary and remove the '0b' prefix
    padded_binary = binary_representation.zfill(8)  # Ensure it has 4 bits by zero-padding
    return padded_binary


def decimal_to_8bits_binary(decimal_number):
    binary_representation = bin(decimal_number)[2:]  # Convert to binary and remove the '0b' prefix
    padded_binary = binary_representation.zfill(8)  # Ensure it has 4 bits by zero-padding
    return padded_binary


def convertBinaryToHexa(binary_number):
    hex_representation = hex(int(binary_number, 2))[2:]
    return hex_representation


def convertstringHexaToInteger(string_hexa):
    int_num = int(string_hexa, 16)
    return int_num


def digitExtraction(hex_number):
    hex_string = hex(hex_number)[2:]  # Convert to hexadecimal string and remove '0x' prefix
    return hex_string


def rotateWindow(window):
    temp = window[0]
    for i in range(3):
        window[i] = window[i + 1]
    window[3] = temp

    return window


def get_value_of_s_box(hexa_num):
    dimensions = hexa_num
    row = convertstringHexaToInteger(dimensions[0])
    column = convertstringHexaToInteger(dimensions[1])
    return s_box[row][column]


def G_for3th_Window(lastColumnInKeyMatrix, round_num):
    third_column = copy.copy(lastColumnInKeyMatrix)
    rotated_column = rotateWindow(third_column)
    galois_column = []
    for i in rotated_column:
        galois_column.append(get_value_of_s_box(i))
    # print(galois_column)
    most_left_byte_binary = hexadecimal_to_8bits_binary(galois_column[0])
    round_num_binary = RC[round_num]
    new_most_left_byte = ''

    for i in range(8):
        xor_bit = xor(int(most_left_byte_binary[i]), int(round_num_binary[i]))
        new_most_left_byte += str(int(xor_bit))

    left_byte_xor_RC = convertBinaryToHexa(new_most_left_byte)

    galois_column[0] = left_byte_xor_RC

    return galois_column


def fillHexaTo2byte(hexa):
    padded = ''
    if len(hexa) < 2:
        padded = hexa.zfill(2)
    else:
        padded = hexa
    return padded


def xorBetweenTwoWindows(window1, window2):
    new_window = []
    for i in range(4):
        byte_of_window1 = hexadecimal_to_8bits_binary(window1[i])
        byte_of_window2 = hexadecimal_to_8bits_binary(window2[i])
        xor_result = ''
        for j in range(8):
            xor_bit = xor(int(byte_of_window1[j]), int(byte_of_window2[j]))
            xor_result += str(int(xor_bit))
        hexa_xor_result = convertBinaryToHexa(xor_result)
        padded = ''
        if len(hexa_xor_result) < 2:
            padded = hexa_xor_result.zfill(2)
        else:
            padded = hexa_xor_result
        new_window.append(padded)
    return new_window


def transposeMatrix(matrix):
    transposed_list = [list(row) for row in zip(*matrix)]
    return transposed_list


def keyGenerationProcess(windows, round_num):
    new_key_matrix = []

    # windows = transposeMatrix(initialKeyMatrix)
    third_column = copy.copy(windows[3])
    galois_col = G_for3th_Window(third_column, round_num)

    new_key_matrix.append(xorBetweenTwoWindows(windows[0], galois_col))
    new_key_matrix.append(xorBetweenTwoWindows(new_key_matrix[0], windows[1]))
    new_key_matrix.append(xorBetweenTwoWindows(new_key_matrix[1], windows[2]))
    new_key_matrix.append(xorBetweenTwoWindows(new_key_matrix[2], windows[3]))

    return new_key_matrix

def generateAllKeys(initial_key_matrix):
    windows = transposeMatrix(initial_key_matrix)
    all_keys = [windows]
    for i in range(10):
        new_key_matrix_transposed = keyGenerationProcess(windows, i)
        all_keys.append(transposeMatrix(new_key_matrix_transposed))
        windows = new_key_matrix_transposed

    return all_keys

In [21]:
# --------- make the matrix --------------
def make_matrix(data):
    key_hex = binascii.hexlify(data.encode()).decode()
    print(key_hex)
    data_matrix = np.empty((4, 4), dtype='object')
    index = 0
    for i in range(4):
        for j in range(4):
            data_matrix[j, i] = key_hex[index] + key_hex[index+1]
            index += 2

    # print(key_matrix) 
    return data_matrix 

key = "qwertyuiazsxdcfv"
key_matrix = make_matrix(key)
print(key_matrix)

print("___________________________")

plainText =  "asdfghjklzxcvbnm"
plainText_matrix = make_matrix(plainText)
print(plainText_matrix)

7177657274797569617a737864636676
[['71' '74' '61' '64']
 ['77' '79' '7a' '63']
 ['65' '75' '73' '66']
 ['72' '69' '78' '76']]
___________________________
6173646667686a6b6c7a786376626e6d
[['61' '67' '6c' '76']
 ['73' '68' '7a' '62']
 ['64' '6a' '78' '6e']
 ['66' '6b' '63' '6d']]


In [11]:
# -------- XORing the plaintext matrix with the key matrix -------------------
def add_key(plainText_matrix , key_matrix):

    matrix1 = np.empty((4, 4), dtype='object')
    
    for i in range(4):
        for j in range(4):
            plain_binary = bin(int(plainText_matrix[i][j], 16))[2:]
            key_binary = bin(int(key_matrix[i][j], 16))[2:]

            matrix1[i][j] = hex(int(plain_binary, 2) ^ int(key_binary, 2) )[2:] 

            if len(matrix1[i][j]) == 1:
                matrix1[i][j] = '0' + matrix1[i][j]

    return matrix1

XORing_matrix = add_key(plainText_matrix , key_matrix)
print(XORing_matrix)


[['10' '13' '0d' '12']
 ['04' '11' '00' '01']
 ['01' '1f' '0b' '08']
 ['14' '02' '1b' '1b']]


In [12]:
# ---------------- Byte Substitution Function -----------------
def Byte_Substitution(matrix1):

    S_B = np.array([['63', '7C', '77', '7B', 'F2', '6B', '6F', 'C5', '30', '01', '67', '2B', 'FE', 'D7', 'AB', '76'],
                    ['CA', '82', 'C9', '7D', 'FA', '59', '47', 'F0', 'AD', 'D4', 'A2', 'AF', '9C', 'A4', '72', 'C0'],
                    ['B7', 'FD', '93', '26', '36', '3F', 'F7', 'CC', '34', 'A5', 'E5', 'F1', '71', 'D8', '31', '15'],
                    ['04', 'C7', '23', 'C3', '18', '96', '05', '9A', '07', '12', '80', 'E2', 'EB', '27', 'B2', '75'],
                    ['09', '83', '2C', '1A', '1B', '6E', '5A', 'A0', '52', '3B', 'D6', 'B3', '29', 'E3', '2F', '84'],
                    ['53', 'D1', '00', 'ED', '20', 'FC', 'B1', '5B', '6A', 'CB', 'BE', '39', '4A', '4C', '58', 'CF'],
                    ['D0', 'EF', 'AA', 'FB', '43', '4D', '33', '85', '45', 'F9', '02', '7F', '50', '3C', '9F', 'A8'],
                    ['51', 'A3', '40', '8F', '92', '9D', '38', 'F5', 'BC', 'B6', 'DA', '21', '10', 'FF', 'F3', 'D2'],
                    ['CD', '0C', '13', 'EC', '5F', '97', '44', '17', 'C4', 'A7', '7E', '3D', '64', '5D', '19', '73'],
                    ['60', '81', '4F', 'DC', '22', '2A', '90', '88', '46', 'EE', 'B8', '14', 'DE', '5E', '0B', 'DB'],
                    ['E0', '32', '3A', '0A', '49', '06', '24', '5C', 'C2', 'D3', 'AC', '62', '91', '95', 'E4', '79'],
                    ['E7', 'C8', '37', '6D', '8D', 'D5', '4E', 'A9', '6C', '56', 'F4', 'EA', '65', '7A', 'AE', '08'],
                    ['BA', '78', '25', '2E', '1C', 'A6', 'B4', 'C6', 'E8', 'DD', '74', '1F', '4B', 'BD', '8B', '8A'],
                    ['70', '3E', 'B5', '66', '48', '03', 'F6', '0E', '61', '35', '57', 'B9', '86', 'C1', '1D', '9E'],
                    ['E1', 'F8', '98', '11', '69', 'D9', '8E', '94', '9B', '1E', '87', 'E9', 'CE', '55', '28', 'DF'],
                    ['8C', 'A1', '89', '0D', 'BF', 'E6', '42', '68', '41', '99', '2D', '0F', 'B0', '54', 'BB', '16']])
    
    matrix2 = np.empty((4, 4), dtype='object')

    for i in range(4):
        for j in range(4):
            row = int(matrix1[i][j][0], 16) 
            column = int(matrix1[i][j][1], 16) 
            matrix2[i][j] = S_B[row][column]

    return matrix2

Byte_Substitution_matrix = Byte_Substitution(XORing_matrix)
print(Byte_Substitution_matrix)

[['CA' '7D' 'D7' 'C9']
 ['F2' '82' '63' '7C']
 ['7C' 'C0' '2B' '30']
 ['FA' '77' 'AF' 'AF']]


In [13]:
# ---------------- shift function ---------------------
def shift(matrix2):
    # Define the shift amounts for each row
    shifts = [0 , 1 , 2 , 3]
    matrix3 = np.empty((4, 4), dtype='object')
    # Shift each row of the matrix by the corresponding amount
    for i in range(matrix2.shape[0]):
        matrix3[i] = np.roll(matrix2[i], -shifts[i])

    # Print the resulting shifted matrix
    # print(f"The resulting shifted matrix is:\n{matrix2}")
    return matrix3

shifted_matrix = shift(Byte_Substitution_matrix)
print(shifted_matrix)

[['CA' '7D' 'D7' 'C9']
 ['82' '63' '7C' 'F2']
 ['2B' '30' '7C' 'C0']
 ['AF' 'FA' '77' 'AF']]


In [14]:
# -------------- Mix Column Function -------------------
def mix_column_function(matrix3):
    Mix_column = np.array([['2', '3', '1', '1'],
                       ['1', '2', '3', '1'],
                       ['1', '1', '2', '3'],
                       ['3', '1', '1', '2']])
    
    matrix4 = np.empty((4, 4), dtype='object')
    for j in range(4):

        for k in range(4):

            initial1 = '00000000'
            for i in range(4):
                # binary = bin(int(test[i][0], 16))[2:]
                binary = format(int(matrix3[i][j], 16), '08b')
                # print(matrix3[i][j])
                # print(binary)
                if Mix_column[k][i] == '2':
                    # print(Mix_column[k][i])
                    # print(binary[0])
                    if binary[0] == 0:
                        binary = binary[1:] + '0'
                    else:
                        binary = binary[1:] + '0'
                        # print(binary)
                        one_B = '00011011' 
                        binary = format(int(binary, 2) ^ int(one_B, 2) , '08b' )
                        # print(binary)

                elif Mix_column[k][i] == '3':
                    binary2 = binary
                    # print(binary2)
                    # print("lllll")
                    if binary[0] == '0':
                        binary = binary[1:] + '0'
                        # print(binary)
                        # print("fhg")
                    else:
                        # print("aaaaaaaa")
                        binary = binary[1:] + '0'
                        one_B = '00011011' 
                        # binary = bin(int(binary, 2) ^ int(one_B, 2) )[2:] 
                        binary = format(int(binary, 2) ^ int(one_B, 2) , '08b' )

                    # binary = bin(int(binary, 2) ^ int(binary2, 2) )[2:] 
                    binary = format(int(binary, 2) ^ int(binary2, 2) , '08b' )

                # initial1 = bin(int(binary, 2) ^ int(initial1, 2) )[2:]
                initial1 = format(int(binary, 2) ^ int(initial1, 2) , '08b' )
                
            #     print("_______________________________________________________")
            # print(initial1)
            val = hex(int(initial1, 2))[2:]
            matrix4[k][j] = val
            # print("hexa " , val)

    return matrix4

mix_column__matrix = mix_column_function(shifted_matrix)
print(mix_column__matrix)
# print(matrix1)

[['96' '8e' '3a' 'eb']
 ['7' 'a' 'c7' 'c2']
 ['ef' '70' 'd1' '4a']
 ['a9' '3b' '97' '37']]


In [22]:

# # Define the state matrix
# state_matrix = np.array([
#     ['63', '7C', '77', '7B'],
#     ['F2', '6B', '6F', 'C5'],
#     ['30', '01', '67', '2B'],
#     ['FE', 'D7', 'AB', '76']
# ])
def max_column1(state_matrix):
    Mix_column = np.array([['2', '3', '1', '1'],
                       ['1', '2', '3', '1'],
                       ['1', '1', '2', '3'],
                       ['3', '1', '1', '2']])
    
    mixed_state = np.zeros_like(state_matrix, dtype=int)
    for col in range(4):
        for row in range(4):
            mixed_state[row][col] = (
                int(Mix_column[row][0] , 16) * int(state_matrix[0][col], 16) ^
                int(Mix_column[row][1] , 16) * int(state_matrix[1][col], 16) ^
                int(Mix_column[row][2] , 16) * int(state_matrix[2][col], 16) ^
                int(Mix_column[row][3] , 16) * int(state_matrix[3][col], 16)
            ) % 256

    # Print the resulting mixed state
    print(f"The resulting mixed state is:\n{mixed_state}")


In [15]:
def shift_inverse(matrix2):
    # Define the shift amounts for each row
    shifts = [0 , 1 , 2 , 3]
    matrix3 = np.empty((4, 4), dtype='object')
    # Shift each row of the matrix by the corresponding amount
    for i in range(matrix2.shape[0]):
        matrix3[i] = np.roll(matrix2[i], shifts[i])

    # Print the resulting shifted matrix
    # print(f"The resulting shifted matrix is:\n{matrix2}")
    return matrix3

shifted_matrix = shift_inverse(Byte_Substitution_matrix)
print(shifted_matrix)

[['CA' '7D' 'D7' 'C9']
 ['7C' 'F2' '82' '63']
 ['2B' '30' '7C' 'C0']
 ['77' 'AF' 'AF' 'FA']]


In [16]:
# ---------------- Byte Substitution Function -----------------
def Byte_Substitution_inverse(matrix1):

    S_B_inverse = np.array([['52', '09', '6A', 'D5', '30', '36', 'A5', '38', 'BF', '40', 'A3', '9E', '81', 'F3', 'D7', 'FB'],
                  ['7C', 'E3', '39', '82', '9B', '2F', 'FF', '87', '34', '8E', '43', '44', 'C4', 'DE', 'E9', 'CB'],
                  ['54', '7B', '94', '32', 'A6', 'C2', '23', '3D', 'EE', '4C', '95', '0B', '42', 'FA', 'C3', '4E'],
                  ['08', '2E', 'A1', '66', '28', 'D9', '24', 'B2', '76', '5B', 'A2', '49', '6D', '8B', 'D1', '25'],
                  ['72', 'F8', 'F6', '64', '86', '68', '98', '16', 'D4', 'A4', '5C', 'CC', '5D', '65', 'B6', '92'],
                  ['6C', '70', '48', '50', 'FD', 'ED', 'B9', 'DA', '5E', '15', '46', '57', 'A7', '8D', '9D', '84'],
                  ['90', 'D8', 'AB', '00', '8C', 'BC', 'D3', '0A', 'F7', 'E4', '58', '05', 'B8', 'B3', '45', '06'],
                  ['D0', '2C', '1E', '8F', 'CA', '3F', '0F', '02', 'C1', 'AF', 'BD', '03', '01', '13', '8A', '6B'],
                  ['3A', '91', '11', '41', '4F', '67', 'DC', 'EA', '97', 'F2', 'CF', 'CE', 'F0', 'B4', 'E6', '73'],
                  ['96', 'AC', '74', '22', 'E7', 'AD', '35', '85', 'E2', 'F9', '37', 'E8', '1C', '75', 'DF', '6E'],
                  ['47', 'F1', '1A', '71', '1D', '29', 'C5', '89', '6F', 'B7', '62', '0E', 'AA', '18', 'BE', '1B'],
                  ['FC', '56', '3E', '4B', 'C6', 'D2', '79', '20', '9A', 'DB', 'C0', 'FE', '78', 'CD', '5A', 'F4'],
                  ['1F', 'DD', 'A8', '33', '88', '07', 'C7', '31', 'B1', '12', '10', '59', '27', '80', 'EC', '5F'],
                  ['60', '51', '7F', 'A9', '19', 'B5', '4A', '0D', '2D', 'E5', '7A', '9F', '93', 'C9', '9C', 'EF'],
                  ['A0', 'E0', '3B', '4D', 'AE', '2A', 'F5', 'B0', 'C8', 'EB', 'BB', '3C', '83', '53', '99', '61'],
                  ['17', '2B', '04', '7E', 'BA', '77', 'D6', '26', 'E1', '69', '14', '63', '55', '21', '0C', '7D']])
    
    matrix2 = np.empty((4, 4), dtype='object')

    for i in range(4):
        for j in range(4):
            row = int(matrix1[i][j][0], 16) 
            column = int(matrix1[i][j][1], 16) 
            matrix2[i][j] = S_B_inverse[row][column]

    return matrix2

Byte_Substitution_matrix = Byte_Substitution_inverse(XORing_matrix)
print(Byte_Substitution_matrix)

[['7C' '82' 'F3' '39']
 ['30' 'E3' '52' '09']
 ['09' 'CB' '9E' 'BF']
 ['9B' '6A' '44' '44']]


In [ ]:

# state_matrix = np.array([
#     ['63', '7C', '77', '7B'],
#     ['F2', '6B', '6F', 'C5'],
#     ['30', '01', '67', '2B'],
#     ['FE', 'D7', 'AB', '76']
# ])

def max_inverse(state_matrix):
    inv_mix_column_matrix = np.array([['0E', '0B', '0D', '09'],
                                      ['09', '0E', '0B', '0D'],
                                      ['0D', '09', '0E', '0B'],
                                      ['0B', '0D', '09', '0E']])

    mixed_state = np.zeros_like(state_matrix, dtype=int)
    for col in range(4):
        for row in range(4):
            mixed_state[row][col] = (
                int(inv_mix_column_matrix[row][0],16) * int(state_matrix[0][col], 16) ^
                int(inv_mix_column_matrix[row][0],16) * int(state_matrix[1][col], 16) ^
                int(inv_mix_column_matrix[row][0],16) * int(state_matrix[2][col], 16) ^
                int(inv_mix_column_matrix[row][0],16) * int(state_matrix[3][col], 16)
            ) % 256

    # Print the resulting mixed state
    print(f"The resulting mixed state is:\n{mixed_state}")


In [20]:
initial_key = "5468617473206D79204B756E67204675"
# Convert the hexadecimal string to bytes
hex_bytes = bytes.fromhex(initial_key)

# Convert the bytes to a string in ASCII format
ascii_string = hex_bytes.decode('utf-8')

# Print the ASCII string
# print("ASCII representation:", ascii_string)




ASCII representation: Thats my Kung Fu
